In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [7]:
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
from collections import Counter
import pickle
from lightgbm import LGBMRegressor
import joblib
from pathlib import Path
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from matplotlib.pyplot import figure
from sklearn.tree import DecisionTreeRegressor

<p style="color:Green;font-size:150%;">1. Data preparation</p>

<p style="background-color:DodgerBlue;color:White;font-size:140%;">1.1 Read csv files</p>

<p style="font-size:120%;">1.11 Code for reducing memory usage</p>

In [8]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
#Referance
#This code snippet is borrowed from https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook

<p style="font-size:120%;">1.12 Read csv files</p>

In [9]:

sales=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
calender=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
evaluation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')

for i in range(1942,1970):
    evaluation['d_'+str(i)]=0
sales=reduce_mem_usage(sales)
calender=reduce_mem_usage(calender)
evaluation=reduce_mem_usage(evaluation)

Memory usage of dataframe is 208.77 MB
Memory usage after optimization is: 45.76 MB
Decreased by 78.1%
Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.19 MB
Decreased by 8.7%
Memory usage of dataframe is 459.43 MB
Memory usage after optimization is: 97.11 MB
Decreased by 78.9%


<p style="font-size:120%;"> 1.13 Evaluation dataframe</p>

In [10]:
print(evaluation.shape)
evaluation.head()

(30490, 1975)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p style="font-size:120%;">1.14 Selling price dataframe</p>

In [11]:
print(sales.shape)
sales.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


<p style="font-size:120%;">1.15 Calender dataframe</p>

In [12]:
print(calender.shape)
calender.head()

(1969, 14)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


<p style="font-size:120%;">1.16 Melting evaluation dataframe</p>

In [13]:

evaluation=pd.melt(evaluation,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='sale')
print(evaluation.shape)
evaluation.head()

(60034810, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


<p style="font-size:120%;">1.17 Merge Calender with evaluation df</p>

In [14]:

evaluation=pd.merge(evaluation,calender,on='d',how ='left')
print(evaluation.shape)
evaluation.head()

(60034810, 21)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


<p style="font-size:120%;">1.18 Merge selling price with evaluation df</p>

In [15]:

evaluation=evaluation.merge(sales,on=['item_id','store_id','wm_yr_wk'],how='left')
print(evaluation.shape)
evaluation.head()

(60034810, 22)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


<p style="font-size:120%;">1.19 Merge a column for days of the month in number with evaluation df</p> 

In [16]:
evaluation['day']=[np.int8(date.split('-')[-1]) for date in evaluation['date']]


<p style="color:Green;font-size:150%;">2. 
Exploratory data analysis (EDA)</p>

<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.1 State wise data analysis</p> 


In [17]:
'''We are grouping data with respect to state_id and we will calculate state 
wise average sales.Then we will create two arrays,one for name of the state
another for average sale.Then we will plot this using Plotly'''
state=evaluation.groupby(['state_id'])['sale'].sum()
labels=['CA','TX','WI']
values=[state['CA'],state['TX'],state['WI']]
#----------------------------------------
fig=make_subplots(rows=1,cols=1,specs=[[{'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels,values=values,name='sales percent'),1,1)
fig.update_traces(hole=0.4,hoverinfo='label+percent+name')
fig.update_layout(title_text='Statewise sales 2011-2016',
                 annotations=[dict(text='sales',x=0.50,y=0.5,font_size=20,showarrow=False)])
fig.show()


In [18]:
'''we are grouping data with respect to state and year and then we will calculate
yearly sales.After that we will get year,state names and percentage sales for
each state on each year.Using this data we will plot a graph'''

state=evaluation.groupby(['state_id','year'])['sale'].sum()
state=pd.DataFrame(state)
year=[i for i in range(2011,2017)]
state=state.sort_values('year')
states=[name[0] for name in state.index.values]
sales=state['sale'].values
splits=[i for i in range(3,31,3)]
percentage={st:[] for st in set(states)}
start=0
index=0
for split in splits:
    total=sum(sales[start:split]) 
    for value in sales[start:split]: 
        percent=np.round((value/total)*100,decimals=2)  
        state_name=states[index]
        percentage[state_name].append(percent)
        index=index+1

    start=split
#---------------------------plot----------------------------- 
fig=go.Figure(go.Bar(x=year,y=percentage['CA'],name='CA',
                     hovertemplate="<br>sales:%{y}%</br>"
                    ))
for state_name in sorted(list(set(states)))[1:]:
    fig.add_trace(go.Bar(x=year,
                         y=percentage[state_name],
                         name=state_name,
                         hovertemplate="<br>Sales:%{y}%</br>"))
    fig.update_layout(title='Yearly percentage sales per state',
                      barmode='stack',
                      xaxis={'categoryorder':'category ascending',
                            'title':'Year'},
                      yaxis={'title':'Percentage sales'}
                     )
fig.show()


1. From the donut chart of sales between 2011 and 2016 for 3 stores,

    * CA has the highest contribution to total sales around 43.6%.

    * TX and WI have approximately equal shares in the total sales.

 
2. From the barplot of year vs percentage sales of the states,

  * Sales contribution of state Wisconsin(WI) is increasing every year which has moved from 24% to 30%.

  * Sales contribution of state Texas(TX) is seen decreasing over the years and its contribution dropped from 30% to 26%.

  * Sales contribution of California(CA) had a slight reduction in its share when it reached from 2011 to 2016.
  
From the above observations we can conclude that California contributes nearly half of the sales and performance of Wisconsin(WI) increases ,while performance of Texas(TX) decreases.


<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.2 Store wise data analysis</p> 



In [19]:
'''We will group the data with respect to store and find store wise sum of sales.
Then we are forming a dataframe for this data and from this we will fetch stores 
and their respected sales as array.We are also forming an array for states 
using store ids and an array for stores performance as rank.Then we are building
a new df using the formed arrays and passing it to Plotly graph function to
plot a bubble plot.
'''
store=evaluation.groupby(['store_id'])['sale'].sum()
store=pd.DataFrame(store)
store=store.sort_values('sale')
stores=[i for i in store.index.values]
sales=store['sale'].values
states=[state.split('_')[0] for state in stores]
sales_rank=[i for i in range(1,11)]
sales_rank=reversed(sales_rank)

store=pd.DataFrame.from_dict({'states':states,'stores':stores,'sales':sales,
                             'rank':sales_rank})
#---------------------------------------------------
fig=px.scatter(store,x='rank',y='sales',size='sales',
              color='stores',hover_name='states',
               title='Sales vs rank',
              log_x=False,size_max=50)
fig.show()

In [20]:
'''we are grouping data with respect to store and year and then we will calculate
yearly sales.After that we will year,store names and percentage sales for
each store on each year.Using this data we will plot a graph.
'''

store=evaluation.groupby(['store_id','year'])['sale'].sum()
store=pd.DataFrame(store)
year=[i for i in range(2011,2017)]
store=store.sort_values('year')
stores=[name[0] for name in store.index.values]
sales=store['sale'].values
splits=[i for i in range(10,61,10)]
percentage={st:[] for st in set(stores)}
start=0
index=0
for split in splits:
    total=sum(sales[start:split]) 
    for value in sales[start:split]: 
        percent=np.round((value/total)*100,decimals=2)  
        store_name=stores[index]
        percentage[store_name].append(percent)
        index=index+1

    start=split
#---------------------------plot----------------------------- 
fig=go.Figure(go.Bar(x=year,y=percentage['CA_1'],name='CA_1',
                     hovertemplate="<br>sales:%{y}%</br>"
                    ))
for store_name in sorted(list(set(stores)))[1:]:
    fig.add_trace(go.Bar(x=year,
                         y=percentage[store_name],
                         name=store_name,
                         hovertemplate="<br>Sales:%{y}%</br>"))
    fig.update_layout(title='Yearly percentage sales per store',
                      barmode='stack',
                      xaxis={'categoryorder':'category ascending',
                            'title':'Year'},
                      yaxis={'title':'Percentage sales'}
                     )
fig.show()

In [21]:
'''We are grouping data with respect to store_id,cat_id and d and will find
sum of the sales for this grouping.Then we will form a df for the grouped data.
After that we will get store,category and day arrays from the df.Using these
array we will form a new df and pass that to Plotly line function to plot the
graph.'''
storewise=evaluation.groupby(['store_id','cat_id','d'])['sale'].sum().astype('float64')
storewise=pd.DataFrame(storewise)
store=[i[0] for i in storewise.index.values]
category=[i[1] for i in storewise.index.values]
day=[int(i[2].split('_')[1]) for i in storewise.index.values]
storewise=pd.DataFrame.from_dict({'Store':store,'Day':day,'Sale':list(storewise['sale'].values),
                                 'Category':category})
storewise=storewise.sort_values('Day')
#-----------------------------------------------------------
fig=px.line(storewise,x='Day',y='Sale',color='Category',facet_col='Store',
           facet_col_wrap=2,
           height=800,
           width=800,
           title='Category wise sales pattern for stores')
fig.update_yaxes(matches=None)


fig.show()

1. From the bubble plot of sales for different stores and their sales contribution rankings,
 
    * CA_1 of California outperformed all other stores.

    * First two of the top performing stores are from california.

    * CA_4 of California is the least performing store.

    * Among the best 5 stores 2 each from CA and WI and one from TX.
 
 
 
2. From the barplot of year vs percentage sales of the stores,
 
    * Among the stores of Wisconsin(WI), contribution of stores WI_3 is decreasing over the year while sales of the WI_2 and WI_1 are increasing.

    * Among the stores of Texas(TX),the contribution of stores TX_1 and TX_2 are seen decreasing while sales of TX_3 remains almost constant.

    * Among the stores of California(CA),
    sales of stores CA_1 and CA_2 have slightly varying contribution on sales across the years.For CA_3 contribution is contracting after 2014 and for CA_4  it remains almost constant.

    * Sales contribution of WI_3 has decreased by approximately 3% while WI_2 has been increased by 6% and WI_1 by 3%. 

    * While the sales contribution of Wisconsin expands each year and the contribution of Texas seems to be contracting.
 
 
 
 
 
3. From the line chart of day vs sale, depicting category wise sales pattern of the sales for different stores for all available days,
 
    * All stores have almost the same sales pattern except stores WI_1 and WI_2.

    * WI_1 and WI_2 exhibit a sudden hike in sales after some periods and it remain constant with no steep fall.

    * The Hobbies and Household sections of all the stores have almost the same sales pattern.
 
 
From the above observations we can conclude that shares of the stores in sales are unevenly distributed,this might be due to location or demography around the stores.While some stores keep on increasing their performance,some stores failed to retain their positions.
 




<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.3 Category wise data analysis</p> 

In [22]:
'''We are grouping the data with respect to cat_id,day and will find the sum
of sales for this grouped data.Then we will form a df for this data and get
category and day data as arrays.Using this arrays we will form new df and pass
it to the Plotly line function to create a plot.
'''

catwise=evaluation.groupby(['cat_id','day'])['sale'].sum()
catwise=pd.DataFrame(catwise)
category=[i[0] for i in catwise.index.values]
day=[i[1] for i in catwise.index.values]
catwise=pd.DataFrame.from_dict({'category':category,'day':day,'sale':list(catwise['sale'].values)})
catwise=catwise.sort_values('day')
#------------------------------------------------
fig=px.line(catwise,x='day',y='sale',color='category',title='Day vs sales')
fig.show()

In [23]:
'''We are grouping data with respect to cat_id and d and calculating sum of 
sales for this grouped data.Then we will form a df for this data and we will
create new arrays of category and day using this df.We will create a df using the
arrays and will pass this data to the Plotly line function to plot a graph.
'''

catwise=evaluation.groupby(['cat_id','d'])['sale'].sum()
catwise=pd.DataFrame(catwise)
category=[i[0] for i in catwise.index.values]
day=[int(i[1].split('_')[1]) for i in catwise.index.values]
catwise=pd.DataFrame.from_dict({'category':category,'day':day,'sale':list(catwise['sale'].values)})
catwise=catwise.sort_values('day')
#------------------------------------------------
fig=px.line(catwise,x='day',y='sale',color='category',title='Sales pattern of categories')
fig.show()

1. From the line plot of day vs sales, depicting average of day wise sales of categories for all available months,

    * Food section has the highest sales followed by the Household and Hobbies sections.

    * Sales for all the sections start to decrease after 15 days of a month.

    * Lowest sale is on the 31st day of a month,probably because there are only 7 months where we have 31 days.


2. From the line plot of day vs sales, depicting sales pattern of categories,

    * All categories maintains almost same sales pattern.

    * Sales have nearly doubled for foods and household sections over the year while for hobbies it's increasing slowly.


From the above observations we can conclude that the Foods section outperformed all other sections.Sales are improving each year for all categories and sales of the categories have a good seasonal pattern.

<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.4 Department wise data analysis</p> 

In [24]:
'''We are grouping the data with respect to dept_id,day and will find the sum
of sales for this grouped data.Then we will form a df for this data and get
department and day data as arrays.Using this arrays we will form new df and pass
it to the Plotly line function to create a plot.
'''

catwise=evaluation.groupby(['dept_id','day'])['sale'].sum()
catwise=pd.DataFrame(catwise)
category=[i[0] for i in catwise.index.values]
day=[i[1] for i in catwise.index.values]
catwise=pd.DataFrame.from_dict({'category':category,'day':day,'sale':list(catwise['sale'].values)})
catwise=catwise.sort_values('day')
#------------------------------------------------
fig=px.line(catwise,x='day',y='sale',color='category',title='Day vs sales')
fig.show()

In [25]:


'''we are grouping data with respect to dept_id and year and then we will calculate
yearly sales.After that we will get year,department names and percentage sales for
each department on each year.Using this data we will plot a graph'''

dept_wise=evaluation.groupby(['dept_id','year'])['sale'].sum()
dept_wise=pd.DataFrame(dept_wise)
year=[i for i in range(2011,2017)]
dept_wise=dept_wise.sort_values('year')
dept=[name[0] for name in dept_wise.index.values]
sales=dept_wise['sale'].values
splits=[i for i in range(7,71,7)]
percentage={st:[] for st in set(dept)}
start=0
index=0
for split in splits:
    total=sum(sales[start:split]) 
    for value in sales[start:split]: 
        percent=np.round((value/total)*100,decimals=2)  
        dept_name=dept[index]
        percentage[dept_name].append(percent)
        index=index+1

    start=split
#---------------------------plot----------------------------- 
fig=go.Figure(go.Bar(x=year,y=percentage['FOODS_1'],name='FOODS_1',
                     hovertemplate="<br>sales:%{y}%</br>"
                    ))
for dept_name in sorted(list(set(dept)))[1:]:
    fig.add_trace(go.Bar(x=year,
                         y=percentage[dept_name],
                         name=dept_name,
                         hovertemplate="<br>Sales:%{y}%</br>"))
    fig.update_layout(title='Yearly percentage sales per state',
                      barmode='group',
                      xaxis={'categoryorder':'category ascending',
                            'title':'Year'},
                      yaxis={'title':'Percentage sales'}
                     )
fig.show()

1. From the line plot of day vs sales, depicting aggregated sales of all departments for all available months,

    * FOODS_3 section outperformed all other departments.It contributes nearly half of the total sales.
    
    * HOBBIES_2 department is the least performing department.The sales remain almost constant throughout the month.
    
    * All departments have almost the same monthly seasonal pattern except HOUSEHOLD_2 and HOBBIES_2.
    
    * Sales for the departments start to decrease after first half of the month.


2. From the barplot of year vs percentage sales, showing year wise percentage sales of all categories,

    * FOODS_3 department has the highest sales contribution and it's seen decreasing each year.

    * Department HOBBIES_2 is the least contributing section and it's share is below 1%.

    * HOUSEHOLD_1 department share in sales are consistently improving every year except in 2014 where it's seen decreased.

    * HOUSEHOLD_2 department has maintained almost constant sales share across the year.

    * HOBBIES_1 department sale is bit inconsistent in its contribution across the year.

    * FOODS_2 department sales contribution showed a decrease in the years 2012 and 2013 then it has improved its performance.

    * FOODS_1 department sales share showed a sudden hike in 2012 ,then it has reduced and remained almost constant across the years.


From the above observations we can conclude that though all the departments have relatively the same sales pattern ,their sales are unevenly distributed.



<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.5 Year wise data analysis</p> 

In [26]:
'''We are grouping data with respect to year and d and calculating the sum
of sales for grouped data.Then we are forming a df using this data and will create
arrays of year and day using this df.We will form a new df with this arrays and
wil pass the df to line function of Plotly to plot the graph.
'''

yearwise=evaluation.groupby(['year','d'])['sale'].sum()
yearwise=pd.DataFrame(yearwise)
year=[i[0] for i in yearwise.index.values]
day=[int(i[1].split('_')[1]) for i in yearwise.index.values]
yearwise=pd.DataFrame.from_dict({'year':year,'day':day,'sale':list(yearwise['sale'].values)})
yearwise=yearwise.sort_values('day')
#---------------------------------------------------
fig=px.line(yearwise,x='day',y='sale',color='year',title='Yearly sales pattern')
fig.show()

In [27]:
'''We are grouping data with respect to year and month and calculating the sum
of sales for grouped data.Then we are forming a df using this data and will create
arrays of year and month using this df.We will form a new df with this arrays and
wil pass the df to line function of Plotly to plot the graph.
'''

yearwise=evaluation.groupby(['year','month'])['sale'].sum().astype('float64')
yearwise=pd.DataFrame(yearwise)
year=[i[0] for i in yearwise.index.values]
month=[i[1] for i in yearwise.index.values]
yearwise=pd.DataFrame.from_dict({'year':year,'month':month,'sale':list(yearwise['sale'].values)})
yearwise=yearwise.sort_values('month')
#---------------------------------------------------
fig=px.line(yearwise,x='month',y='sale',color='year',title='Month vs Sales yearwise'
         )
fig.show()

1. From the line plot of day vs sales, depicting sales pattern of each year,

    * Sales are increasing with same pace every year except the year 2014 where it has slowed down.

    * All the year have the same sales pattern.


2. From the line plot of month vs sale, depicting monthly sales of each year,

    * For 2011 we can see that sales started from very low because we've data only from jan 29 of 2011.

    * From 2011 to 2012 sales increased at a much faster rate and In 2013 it’s speed has nearly halved.

    * In 2014 from the second half of the year the sale has dropped below the same half sale of the year 2013.

    * The sales started to recover in 2015 only in the month of April.

    * In 2016 we had a good beginning and you can see then it's dropping. It may be because of insufficient data for 2016.

From the above observations we can conclude that though sales of each year have same seasonal trend, somtimes they are subjected to undergo unexpected trend.

<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.6 Month wise data analysis</p> 

In [28]:
'''We are grouping data with respect to month and calculating the sum
of sales for grouped data.Then we will create an array for months and we 
will form a new df with this array and sales details.Then we will pass the 
df to bar function of Plotly to plot a barplot.
'''

monthly=evaluation.groupby(['month'])['sale'].sum()
months=['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sept','Oct',
        'Nov','Dec']
monthly=pd.DataFrame.from_dict({'Month':months,'Sales':list(monthly.values)})
fig=px.bar(monthly,x='Month',y='Sales',
          title='Overall monthly sales')
fig.show()

In [29]:

'''We are grouping data with respect to month and day and calculating the sum
of sales for grouped data.Then we are forming a df using this data and will create
arrays of month and day.We will create a new array for months by replacing
numerical values of the month by name of the month.Then we will form a new df 
using month,day and sale arrays and will pass that to a Plotly line function
to plot a graph.
'''

monthwise=evaluation.groupby(['month','day'])['sale'].sum()

monthwise=pd.DataFrame(monthwise)
month=[i[0] for i in monthwise.index.values]
day=[i[1] for i in monthwise.index.values]
months=['Jan','Feb','Mar','Apr','May','Jun','July','Aug','Sept','Oct',
        'Nov','Dec']
m_range=[i for i in range(1,13)]
months=dict(zip(m_range,months))
month=[months[i] for i in month]
monthwise=pd.DataFrame.from_dict({'month':month,'day':day,'sale':list(monthwise['sale'].values)})
monthwise=monthwise.sort_values('day')
#------------------------------------------------
fig=px.line(monthwise,x='day',y='sale',color='month',title='Monthly sales pattern')
fig.show()

1. From the barplot of month vs sales,

    * We have the highest sales on March and lowest on November.

    * The months of February,March,April and May have contributed the highest sales.

    * Months of November,December have the least contribution on sales.

    * Sales increase in the first quarter of year then it reduces when it goes towards second quarter and again increases but slightly for next two months and collapses in the month of september.Then slightly increases in October followed by a sharp fall in november it again increases towards first quarter.


2. From the line plot of day vs sales depicting average sales on each day for all available months,

    * Twenty fifth of december has lowest sales,may be because it's christmas day.Followed by the leap day,February 29 and new year day 1st of january.

    * Third of April has the highest daily average sales followed by February 13 may be because February 14 is valentine's day.

    * January 31 has the highest month end sales followed by february 28 and march 30.

    * Beginning of the last week of November has a sharp fall on it's sales also Month of May has a sharp fall after 22nd May.

    * Sales of all months contracts in the second half of the every month and start to recover at the end of the last week.
    
    
From the above observations we can conclude that From February to May we have top sales and lowest in the last two months of the year.There are a lot of seasonal trends in monthly sales.



<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.7 Weekly data analysis</p> 

In [30]:
'''We are grouping data with respect to weekday and cat_id and calculating the sum
of sales for grouped data.Then we are forming a df using this data and will create
arrays of categories and days using this df.Then we will form a new df with these arrays.
we will pass data as three separate categories to bar function of Plotly to 
plot the graph.
'''

weekday=evaluation.groupby(['weekday','cat_id'])['sale'].sum()
days=[day[0] for day in weekday.keys()]
cat=[cat[1] for cat in weekday.keys()]
weekDayFrame=pd.DataFrame.from_dict({'day':days,'category':cat,'sales':weekday.values})
foods=weekDayFrame[weekDayFrame['category']=='FOODS']
hobbies=weekDayFrame[weekDayFrame['category']=='HOBBIES']
household=weekDayFrame[weekDayFrame['category']=='HOUSEHOLD']

#---------------------------plot-----------------------------
fig=go.Figure(data=[go.Bar(name='FOODS',x=list(foods['day']),y=list(foods['sales'])
                          ),
                   go.Bar(name='HOBBIES',x=list(hobbies['day']),y=list(hobbies['sales'])
                          ),
                   go.Bar(name='HOUSEHOLD',x=list(household['day']),y=list(household['sales'])
                          )])
fig.update_layout(barmode='group',
                  title='Daily sale',
                  xaxis={'title':'Day'},
                  yaxis={'title':'Sale'}
                 )
fig.show()



In [31]:
'''We are grouping data with respect to weekday and day,then calculating 
the sum of sales for grouped data.Later we will create an array for week day and month
we will form a new df with this arrays and sales.Then we will pass the 
df to line function of Plotly to plot a line plot.
'''

weekwise=evaluation.groupby(['weekday','day'])['sale'].sum()
weekwise=pd.DataFrame(weekwise)
weekday=[i[0] for i in weekwise.index.values]
day=[i[1] for i in weekwise.index.values]
weekwise=pd.DataFrame.from_dict({'weekday':weekday,'day':day,'sale':list(weekwise['sale'].values)})
weekwise=weekwise.sort_values('day')
#------------------------------------------------
fig=px.line(weekwise,x='day',y='sale',color='weekday',title='Weekly sales pattern')
fig.show()

1. From the barplot of day vs sale, depicting average sale for each days of a week for all categories with available data,

    * Sales on Foods section is highest on Sunday followed by Saturday.

    * Sales of Hobbies and Household sections are high on Saturday followed by sunday.

    * Sales on Foods and Hobbies sections are lowest on Tuesday.

    * Sales on the Household section is lowest on wednesday.

    * Generally sales increases in the midst of the week days and reaches highest in the weekends and drops in the beginning of the week days.


2. From the line plot of day vs sales, depicting average sales on each day for all available weekdays,

    * Sales are highest when the 6th and 15th of the month falls on Sunday ,7th on saturday.

    * Sometimes sales in the second half of the month for weekends is much higher than first half sales of week days.

    * Wednesday records lowest sales in the first half and second half of the month.


From the above observations we can conclude that sales are highest on weekends and lowest on weekdays. Also, sales drop after the second week of a month.

<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.8 Calender Events data analysis</p> 

2.8.1 SNAP day data analysis

In [32]:
'''We are grouping data with respect to d,cat_id and snap_CA then calculating 
the sum of sales for grouped data.Later we will create an array for day,snap
categories and snap day status.we will form a new df with these arrays and sales.
Then we will pass the df to line function of Plotly to plot a line plot.
'''

snap_cat=evaluation.groupby(['d','cat_id','snap_CA'])['sale'].sum()

day=[int(i[0].split('_')[1]) for i in snap_cat.keys()]
snap_categ=[i[1] for i in snap_cat.keys()]
snap_state=[i[2] for i in snap_cat.keys()]

sale=snap_cat.values
snap_cat=pd.DataFrame.from_dict({'day':day,'snap_categ':snap_categ,'snap_state':
                                snap_state
                                ,'sale':sale})
snap_cat=snap_cat.sort_values('day')
#------------------------------------------
fig=px.line(snap_cat,x='day',y='sale',color='snap_state',facet_col='snap_categ',
           facet_col_wrap=1,
           height=800,
           width=800,
           title='Category wise sales pattern for stores')
fig.update_yaxes(matches=None)
fig.show()

2.8.2 Event days data analysis

In [33]:
'''We are calculating sum of sales for each events by combining both columns
events and grouping them by eventname. '''
event_sum1=evaluation[['event_name_1','sale']]
event_sum2=evaluation[['event_name_2','sale']]
event_sum2=event_sum2.rename(columns={'event_name_2':'event_name_1'})
event_sum1=event_sum1.append(event_sum2)
event_sum1=event_sum1.groupby(['event_name_1'])['sale'].sum()

'''We are dropping columns with nan values from event1 and combining days
and event name to get unique events on each day.
'''
event1=evaluation[['d','event_name_1']].dropna()
event1=set(event1['d'].astype(str)+'#'+event1['event_name_1'].astype(str))
event1=[i.split('#')[1] for i in event1]

'''We are dropping columns with nan values from event2 and combining days
and event name to get unique events on each day.
'''
event2=evaluation[['d','event_name_2']].dropna()
event2=set(event2['d'].astype(str)+'#'+event2['event_name_2'].astype(str))
event2=[i.split('#')[1] for i in event2]

'''Now we will combine events from each events columns and compute number of 
occurrences of the events using the Counter.
'''
event1.extend(event2)
event1=Counter(event1)

'''We are calculating average sales for each events using previously calculated
event sum and event counts.'''
events=list(event1.keys())
avg_event1=[event_sum1[event]/event1[event] for event in events]

'''We will find types of the events by forming a dict using event_name and
type.'''
event=evaluation['event_name_1'].dropna()
types=evaluation['event_type_1'].dropna()
event_dict=dict(zip(list(event),list(types)))
event_type=[event_dict[event] for event in events]

'''We are now calculating overall average of sales for all days including events
and appending that to events array with event name as Overall avg and event type as 
General.Now we will form a df using all  the arrays and pass it to bar function
of Plotly to plot a barplot.
'''
overal_avg=evaluation.groupby(['d'])['sale'].sum().mean()
avg_event1.insert(0,overal_avg)
events.insert(0,'Overall avg')
event_type.insert(0,'General')

event1df=pd.DataFrame.from_dict({'events':events,'event_type':event_type,
                                 'avg_sale':avg_event1})

#---------------------------------------------
fig=px.bar(event1df,x='events',y='avg_sale',color='event_type', 
          title='Avg event sales vs avg sales')
fig.show()

1. From the line plot of day vs sale, depicting normal days and SNAP days sales of store CA for all categories,

    * For most of the time on SNAP days, sales in the FOODS section increases compared to previous day of SNAP day.

    * Though SNAP is not applicable on Hobbies and Household items, their sales rise on SNAP days.





2. From the barplot of events vs avg sale, depicting events and average sales on that days and overall sales average,

    * Among the events, Christmas day has lowest average sale may be store was not opened on that day.

    * Thanksgiving day has the second lowest sales may be due to people celebrating in the home and they might have completed their shopping before that day.

    * New year day records low average sales as well ,this might be due to people spending their time in home after a long night's celebrations.

    * Overall on national holidays stores performance is poor except on Labor day on which the highest average sale is recorded among all events.

    * Among religious festivals on the days of lent start and lent week stores record low average sales.This may be because people spending their time in fasting or prayer on these days.

    * On other religious holidays all stores exhibit an average performance except on orthodox Easter day on which the third highest average sales among the events is observed.

    * Among the cultural festival on Halloween day a slight drop in average sale can be observed,this may be due to people spend their time on halloween activities for parties.

    * On Father's day and Easter's day sales go slightly up compared to average sale.

    * Among the sporting events on Super Bowl's day we have the second highest sales average among the events and NBF final day's sales are slightly below average.



From our observations we can conclude that,on most of the holidays we have an average sales.For some events we have much lower sales than average sale,this might be due to people spent their time in home on those days.There are only three to four events which give considerable rise in the sales which might be due to people opted shopping before the events day.On SNAP days we have considerable rise in sales in all categories.Events features with some lag would be useful.

<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.9 Selling price analysis</p> 

In [34]:
''' We will select a random item id name and form a df with only this item id
then will create array for store_id and item_id.Then we will fetch wm_yr_wk,that
is week ids and will convert them as numbers ranging from 1 to len(wm_yr_wk) 
with the help of a dictionary.Finally we will form a df using items,stores,week id,
selling price and the sales.This df will be passed to line function of Plotly to 
plot the line plot.
'''
items=evaluation[evaluation['item_id']=='FOODS_3_551']
item=items['item_id'].values.astype(object)
store=items['store_id'].values.astype(object)
wk=sorted(list(set(items['wm_yr_wk'])))
wk=dict(zip(wk,list(range(1,len(wk)+1))))
week=items['wm_yr_wk'].values
week=[wk[i] for i in week]
sp=np.float64(items['sell_price'].values)
sale=np.float64(items['sale'].values)
items=pd.DataFrame.from_dict({'item':item,'week':week,'sp':sp,'store':store,
                             'sale':sale})

#-----------------------------------
fig=px.line(items,x='week',y='sale',color='sp',facet_col='store',
           facet_col_wrap=2,
           height=800,
           width=800,
           title='Selling price vs sale random items')
fig.update_yaxes(matches=None)
fig.show()

From the line plot of week vs sale, depicting selling prices for a randomly selected item on each week and for each stores,

* The selling price of the item increases with time.

* Sometimes we can see  the item being sold for lower price compared to their regular price,this may be due to store promotions.

* In some stores the item was not available for a long period.

* The regular selling price among stores are centralized but the stores do have a separate promotion price such as on week 116 of TX_1 and week 181 of TX_2.

* Increase in selling price doesn't really affect the sales.

* In some stores the item movement is slow and in some stores it's fast moving.


From the above observations we can conclude that the selling price changes with time period and it's almost constant for all the stores.Increase in selling price might not affect sales.


<p style="background-color:DodgerBlue;color:White;font-size:140%;">2.10 Zero sale analysis</p> 

In [35]:
'''We will group the data with respect to item_id and year and calculate
the sum of sales.Then we will form arrays of item_id,year and sales.Using 
these arrays will form a df and will select only items with zero sales.
We will group the df with year and will calculate the number of items with
zero sales using count().Then we will form a new df of year and no_sale_count 
and passing that to a Plotly line function to plot a line plot.
'''

no_sale=evaluation.groupby(['item_id','year'])['sale'].sum()
item=[i[0] for i in no_sale.keys()]
year=[i[1] for i in no_sale.keys()]
sale=no_sale.values
no_sale=pd.DataFrame.from_dict({'item':item,'year':year,'sale':sale})
no_sale=no_sale[no_sale['sale']==0]
no_sale=no_sale.groupby(['year'])['sale'].count()

no_sale=pd.DataFrame.from_dict({'year':no_sale.keys(),
                                 'no_sale_count':no_sale.values})
#-------------------------------------------
fig=px.line(no_sale,x='year',y='no_sale_count',
           title='No sale items over the year')
fig.update_yaxes(matches=None)
fig.show()


In [36]:
'''Forming quarterly feature to check the quarterly zero sales counts'''
quarter={'01':1,'02':1,'03':1,'04':2,'05':2,'06':2,'07':3,'08':3,'09':3,'10':4,'11':4,'12':4}
evaluation['quarter']=[np.int8(quarter[date.split('-')[1]]) for date in evaluation['date']]

In [37]:
'''We will group the data with respect to item_id,year and quarter and calculate
the sum of sales.Then we will form arrays of item_id,year,quarter and sales.Using these arrays will form a df and will select only items with zero sales.
We will group the df with year and will calculate the number of items with
zero sales using count().Then we will form a new df of year,quarter and no_sale_count 
and passing that to a Plotly line function to plot a line plot.
'''

no_sale=evaluation.groupby(['item_id','year','quarter'])['sale'].sum()
item=[i[0] for i in no_sale.keys()]
year=[i[1] for i in no_sale.keys()]
quarter=[i[2] for i in no_sale.keys()]
sale=no_sale.values
no_sale=pd.DataFrame.from_dict({'item':item,'year':year,'quarter':quarter,
                                'sale':sale})
no_sale=no_sale[no_sale['sale']==0]
no_sale=no_sale.groupby(['year','quarter'])['sale'].count()

quarter=[i[1] for i in no_sale.keys()]
year=[i[0] for i in no_sale.keys()]

no_sale=pd.DataFrame.from_dict({'year':year,'quarter':quarter,
                                 'no_sale_count':no_sale.values})
#-------------------------------------------
fig=px.line(no_sale,x='quarter',y='no_sale_count',
            facet_col='year',
           facet_col_wrap=2,
           height=800,
           width=800,
           title='Quarterly no sale items')
fig.update_yaxes(matches=None)
fig.show()

1. From the line plot of no_sale_count vs year, depicting number of yearly unsold items,

    * In 2011 we have a lot of items remains unsold.

    * In 2012 not selling items count has been reduced to nearly half from 2011's count.

    * In 2013 not selling items count further halved.

    * In 2014 not selling items count reduced to 30 and then 2 on 2015 and 1 on 2016.

    * It seems that most of the items provided in the dataset were not actually available in the store in the beginning years and then they were introduced slowly.

2. From the line plot of no_sale_count vs year, depicting number quarterly unsold items for each year,

    * Item with no sales as observed from the yearly no sale plot is continuously reducing on each quarter except in the year 2015 which shows some uneven fluctuation in quarterly no sale counts.

    * The no-sale counts for 2016 increasing from quarter 2 is due to unavailability of data.
    

I thought that we could build a feature for quarterly no sale counts but from the observation we can conclude that the counts are not stable,they keep reducing.So it will not be a useful feature.






<p style="color:Green;font-size:150%;">3. 
Feature engineering</p>

<p style="background-color:DodgerBlue;color:White;font-size:140%;">3.1 Encoding of categorical features</p>

In [ ]:
del evaluation,calender,sales
gc.collect()

In [4]:
'''Read evaluaton file and combine with calender and date and remove unwanted 
features'''
sales=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
calender=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
evaluation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
for i in range(1942,1970):
        evaluation['d_'+str(i)]=0


sales=reduce_mem_usage(sales)
calender=reduce_mem_usage(calender)
evaluation=reduce_mem_usage(evaluation)

evaluation=pd.melt(evaluation,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='sale')
evaluation=pd.merge(evaluation,calender,on='d',how ='left')
evaluation=evaluation.merge(sales,on=['item_id','store_id','wm_yr_wk'],how='left')

'''Add day featue and remove unwanated columns'''
evaluation['day']=[np.int8(date.split('-')[-1]) for date in evaluation['date']]
evaluation=evaluation.drop(['date','weekday','event_type_1','event_type_2'],axis=1)
#sale,'event_type_1',

Memory usage of dataframe is 208.77 MB
Memory usage after optimization is: 45.76 MB
Decreased by 78.1%
Memory usage of dataframe is 0.21 MB
Memory usage after optimization is: 0.19 MB
Decreased by 8.7%
Memory usage of dataframe is 459.43 MB
Memory usage after optimization is: 97.11 MB
Decreased by 78.9%


In [5]:

'''Function to encode dataframe catagory to numericals'''
def form_cat(x):
    return dicts[x]
'''Function to remove d from date'''
def form_day(x):
    return x.split('_')[1]   

'''get unique categorical values from category features of df'''
cats=['id','state_id','store_id','cat_id','dept_id','item_id','event_name_1',
     'event_name_2']
for cat in cats:
    if (cat=='event_name_1') | (cat=='event_name_2'):
        evaluation[cat]=evaluation[cat].astype(object).fillna(0)
        vals=set(evaluation[cat].values) 
        cat_dic=dict(zip(vals,list(range(-1,len(vals)-1))))
    else:
        vals=set(evaluation[cat].values)   
        cat_dic=dict(zip(vals,list(range(len(vals)))))
    joblib.dump(cat_dic,cat+'.pkl')
    dicts=cat_dic
    evaluation[cat]=evaluation[cat].apply(lambda x:dicts[x]).astype(np.int16)
evaluation['d']=evaluation.d.apply(lambda x:x.split('_')[1]).astype(np.int16)
evaluation=reduce_mem_usage(evaluation)
evaluation.to_pickle('/kaggle/working/evaluation.pkl')

Memory usage of dataframe is 2290.15 MB
Memory usage after optimization is: 1946.62 MB
Decreased by 15.0%


<p style="background-color:DodgerBlue;color:White;font-size:140%;">3.1 Event features with lag</p>

   When the features are correlated to their previous time step values, we can construct new features from them by introducing some lag to them .That is shifting the values from  a time step to forward with constant step until reaching the last point.We can use shift( ) in pandas library to create the lags.

In [23]:
evaluation['lagca_5']=evaluation.groupby(['id','item_id','dept_id','cat_id','store_id','state_id'])['snap_CA'].shift(5).astype(np.float16)
evaluation['lagtx_5']=evaluation.groupby(['id','item_id','dept_id','cat_id','store_id','state_id'])['snap_TX'].shift(5).astype(np.float16)
evaluation['lagwi_5']=evaluation.groupby(['id','item_id','dept_id','cat_id','store_id','state_id'])['snap_WI'].shift(5).astype(np.float16)


<p style="background-color:DodgerBlue;color:White;font-size:140%;">3.2 Minimum price and standard deviation of price</p>

Standard deviation measures the amount of variation in a distribution's values.A low standard deviation indicates values are concentrated near to mean while high standard deviation indicates values are spread in a wide range.We can use std() of pandas to measure standard deviation of time series in a given window.



In [ ]:
evaluation['price_min']=evaluation.groupby(['store_id','item_id'])['sell_price'].transform('min')
evaluation['price_std']=evaluation.groupby(['store_id','item_id'])['sell_price'].transform('std')

In [ ]:
evaluation

<p style="color:Green;font-size:150%;">4. 
Modelling</p>
We will train the model categorywise and combine the predictoion to form final submission.

<p style="font-size:120%;">4.11 Useful functions</p>

In [6]:
     '''Split the data'''
def split_data(evaluation):
    evaluation=evaluation.dropna()
    train=evaluation[evaluation['d']<=1913]
    valid=evaluation[(evaluation['d']>1913) & (evaluation['d']<=1941)]
    test=evaluation[evaluation['d']>1941]
    train_y=train['sale']
    valid_y=valid['sale']
    test_y=test['sale']
    train=train.drop(['sale'],axis=1)
    valid=valid.drop(['sale'],axis=1)
    test=test.drop(['sale'],axis=1)
    return train,train_y,valid,valid_y,test,test_y


    '''Predict using trained lightgbm model.'''
def predict_model(valid,model,model_lev='linear'): 
    val=valid.copy()
    path=os.getcwd()
    model_lev=model_lev
    model=joblib.load(path+'/'+model_lev+'/'+"model"+str(model)+".pkl")
    val['pred_sale']=model.predict(valid)
    gc.collect()                               
    return val



'''RMSE score function'''
def rmse(actual,pred):
    return np.sqrt(np.sum(np.square(pred-actual))/len(actual))



In [7]:
'''Function to plot feature importance'''
def plot_importance(cols,model_lev='linear',cats=[0,1,2]):
    path=os.getcwd()
    model_lev=model_lev
    for i in range(1,4):
        model=joblib.load(path+'/'+model_lev+'/'+"model"+str(cats[i])+".pkl")
        features=cols

        if model_lev=='linear':
            importances=model.coef_
        else:
            importances=model.feature_importances_   

        indices= (np.argsort(importances))[-50:]
        figure(figsize=(6,4), dpi=80)
        plt.title('Feature Importances')
        plt.barh(range(len(indices)), importances[indices], color='r', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.xlabel('Importance')
        plt.show()



In [8]:
def form_submission(val,data_type):

    ''' Using id dictionary cahange the id name and from final submission file using pivot function'''

    val=val[['id','d','pred_sale']]
    warnings.filterwarnings('ignore')
    id_dict=pd.read_pickle('id.pkl')
    id_dict=dict(zip(list(id_dict.values()),list(id_dict.keys())))
    dicts=id_dict
    val['id']=val['id'].apply(lambda x:dicts[x])
    if data_type=='_validation':
         val['id']= val['id'].apply(lambda x:'_'.join(x.split('_')[:-1])+data_type)
        

    val=val.pivot(columns='d',values='pred_sale',index='id')
    val=val.rename_axis(None, axis=1)
    val=val.reset_index()
    return val

In [9]:
'''Parms for lightGBM'''
params={
        'n_estimators':1000,
        'learning_rate':0.3,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'max_depth':8,
        'num_leaves':50,
       'min_child_weight':300
}
'''Function to train model and get predictions'''
def get_predictions(evaluation,model_lev='linear',params=params):
    test_frame=pd.DataFrame()
    valid_frame=pd.DataFrame()
    true_y=[]
    cols=[]
    cats=set(evaluation['cat_id'].values)
    i=0
    '''For each category split the data and train with requested model'''
    for cat in cats:
        train,train_y,valid,valid_y,test,test_y=split_data(evaluation[evaluation['cat_id']==cat])
        i=i+1
        path=os.getcwd()
        model_lev=model_lev
        if not os.path.exists(model_lev):
            os.mkdir(model_lev)
        if  model_lev=='linear':
            model=LinearRegression()
            model.fit(train,train_y)
        elif model_lev=='tree':
            model=DecisionTreeRegressor(max_depth=14,random_state=42)
            model.fit(train,train_y)
        elif model_lev=='boosting':
            model=LGBMRegressor(deterministic=True,**params)
            model.fit(train,train_y,eval_set=[(train,train_y),(valid,valid_y)],verbose=20
                    ,early_stopping_rounds=20,eval_metric ="rmse")
        '''After training the model store it in a folder'''
        joblib.dump(model,path+'/'+model_lev+'/'+'model'+str(cat)+".pkl")
        gc.collect()
        '''get predictions for the validation and test data'''
        val=predict_model(valid,model=cat,model_lev=model_lev)
        valid_frame=valid_frame.append(val)
        true_y.extend(list(valid_y.values))


        val=predict_model(test,model=cat,model_lev=model_lev)
        test_frame=test_frame.append(val)
        cols=list(evaluation.columns)
    '''Prepare submission format for valid and test output and return along with actual
    ,predicted values of valid data and feature names'''
    pred_y=valid_frame['pred_sale']
    df=form_submission(valid_frame,data_type='_validation')
    test_df=form_submission(test_frame,data_type='_evaluation')
    return pred_y,true_y,df,test_df,cols

In [ ]:
test_frame=pd.DataFrame()
valid_frame=pd.DataFrame()

cats=set(evaluation['cat_id'].values)
'''For each category split the data and train with requested model'''
for cat in cats:
    train,train_y,valid,valid_y,test,test_y=split_data(evaluation[evaluation['cat_id']==cat])

    model=LinearRegression()
    model.fit(train,train_y)

    '''get predictions for the validation and test data'''
    val=model.predict(valid)
    valid_frame=valid_frame.append(val)

    val=predict_model.predict(test)
    test_frame=test_frame.append(val)

<p style="background-color:DodgerBlue;color:White;font-size:140%;">4.1 Linear Regression</p>
It's a linear model and it will try to find a plane which will best fits the given data.


In [ ]:
'''Model prediction and loss'''
pred_y,true_y,df,test_df,cols=get_predictions(evaluation,model_lev='linear',params=params)

print('RMSE for linear model is:',rmse(true_y,pred_y))

In [ ]:
'Feature importance'
plot_importance(cols,model_lev='linear')

<p style="background-color:DodgerBlue;color:White;font-size:140%;">4.2 Decision Tree Regressor </p>
Decision tree has a tree like structures where data is divided as root nodes where the tree begins,decision nodes where again nodes will be splitted and subnodes are formed based on conditon, and finally the leaf nodes they are the nodes which will not be divided again. 


In [ ]:
'''Model prediction and loss'''
pred_y,true_y,df,test_df,cols=get_predictions(evaluation,model_lev='tree',params=params)
print('RMSE for decision tree model is:',rmse(true_y,pred_y))

In [ ]:
'Feature importance'
plot_importance(cols,model_lev='tree')

<p style="background-color:DodgerBlue;color:White;font-size:140%;">4.3 lightGBM</p>


In [ ]:

'''Model prediction and loss'''
pred_y,true_y,df,test_df,cols=get_predictions(evaluation,model_lev='boosting',params=params)
print('RMSE for boosting model is:',rmse(true_y,pred_y))

In [ ]:
'Feature importance'
plot_importance(cols,model_lev='boosting')

<p style="background-color:DodgerBlue;color:White;font-size:140%;">5.1 Submission</p>


In [ ]:
'''Final file for submission'''
columns=['id']+['F'+str(i) for i in range(1,29)]
df.columns=columns
test_df.columns=columns
df=df.append(test_df).reset_index().drop(['index'],axis=1)
df.to_csv('submission.csv',index=False)

<p style="background-color:DodgerBlue;color:White;font-size:140%;">5.2 Results</p>


In [ ]:
from prettytable import PrettyTable 
table=PrettyTable()
table.field_names=(["Model","RMSE",'Private_score'])
table.add_row(["Linear_regression","3.555","1.59598"])
table.add_row(["Decision Tree","2.6232","0.87011"])
table.add_row(["lightGBM","2.2423","0.57798"])
table

<img src='https://i.imgur.com/4NC3OIz.jpg'>
<img src='https://i.imgur.com/QOQMukh.jpg'>

<p style="background-color:DodgerBlue;color:White;font-size:140%;">5.3 Observations</p>

At first i've only focused on sales based features i've created features using rolling window,expanding window,skewness,kurtosis,detrended features ..etc,i found that features based on sales increasing error.When i’ve re-checked the kaggle codes i’ve found that a solution( https://www.kaggle.com/code/jagdmir/m5-forecasting-part-two-lgbm-regressor#Create-Train,Validity-and-Test-Dataframes )only with given features gives a private score of 0.68102.I’ve tried to improve it by adding new features. I've tried SNAP days with lags, a lag of 5 days with SNAP days improved the result significantly,day of the month,min selling price and standard deviation of the price have improved the result further.


In [ ]:
#Should be on another ipynb for validation /Final pipeline


'''form dataframe using input data and pre-processing'''
def preprocess(inputs):
    _id=inputs[0]+'_'+inputs[3]+'_evaluation'
    inputs=[_id]+inputs
    inputs=[[i] for i in inputs]
    evaluation=pd.DataFrame.from_dict(dict(zip(['id','item_id','dept_id','cat_id','store_id',
                                       'state_id','d','wm_yr_wk','date','wday','month','year'
                                                ,'event_name_1','event_name_2','snap_CA','snap_TX',
       'snap_WI','sell_price'],inputs)))

    evaluation['day']=[np.int8(date.split('-')[-1]) for date in evaluation['date']]


    '''get unique categorical values from category features of df'''
    cats=['id','state_id','store_id','cat_id','dept_id','item_id','event_name_1',
         'event_name_2']
    for cat in cats:
        if (cat=='event_name_1') | (cat=='event_name_2'):
            evaluation[cat]=evaluation[cat].astype(object).fillna(0)
       
        dicts=pd.read_pickle(cat+'.pkl')

        evaluation[cat]=evaluation[cat].apply(lambda x:dicts[x]).astype(np.int16)

    evaluation['d']=evaluation.d.apply(form_day).astype(np.int16)
    evaluation['sell_price']=evaluation['sell_price'].astype(np.float16)
    evaluation=evaluation.drop('date',axis=1)
    

    evaluation['lagca_5']=0.0
    evaluation['lagtx_5']=0.0
    evaluation['lagwi_5']=0.0
    evaluation['price_min']=0.0
    evaluation['price_std']=0.0
    return evaluation


def decode(val):

    ''' decode id values and reduce the dataframe to only id and sale.'''

    val=val[['id','d','pred_sale']]
    warnings.filterwarnings('ignore')
    id_dict=pd.read_pickle('id.pkl')
    id_dict=dict(zip(list(id_dict.values()),list(id_dict.keys())))
    dicts=id_dict
    val['id']=val['id'].apply(lambda x:dicts[x])

    val['id']= val['id'].apply(lambda x:'_'.join(x.split('_')[:-1])+'_evaluation')
    val=val.pivot(columns='d',values='pred_sale',index='id')
    val=val.rename_axis(None, axis=1)
    val=val.reset_index()

    return val



'''Predict for the given 1D input'''
def predict(inputs):
    cat=inputs[3]
    evaluation=preprocess(inputs)

    if cat=='FOODS':
        model=joblib.load('model0.pkl')
    elif cat=='HOUSEHOLD':
        model=joblib.load('model1.pkl')
    else:
        model=joblib.load('model2.pkl')
    evaluation['pred_sale']=model.predict(evaluation)
    evaluation=decode(evaluation)
    return evaluation

In [ ]:
inputs=['FOODS_3_668','FOODS_3','FOODS','CA_1','CA','d_1970',11621,'2016-06-20',2,1,2016
                                                ,np.nan,np.nan,0,0,0,'1.480469']
sales=predict(inputs)
print('prediction of sales for given input,\n\n',sales)